In [1]:
# Run in python console
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srisuma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd

data_full=pd.read_csv('Amazon_Unlocked_Mobile.csv',error_bad_lines=False)

In [3]:
data_text = data_full[['Reviews']]
data_text=data_text.iloc[:400000]

In [4]:
pprint(data_text)

                                                  Reviews
0       I feel so LUCKY to have found this used (phone...
1       nice phone, nice up grade from my pantach revu...
2                                            Very pleased
3       It works good but it goes slow sometimes but i...
4       Great phone to replace my lost phone. The only...
...                                                   ...
399995  Very disappointed... I had trouble sending pic...
399996  I love all the features that this phone has. I...
399997  pro:good lightingdecent battery lifecon:slow p...
399998  I could never use the phone: be careful with w...
399999                                         good phone

[400000 rows x 1 columns]


In [5]:
data_text.head()
data_text=data_text.dropna()
print(len(data_text))

399938


In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','phone','cellphone','smartphone','be','randomly','mom'])


In [7]:
# Convert to list
data = data_text.Reviews.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['I feel so LUCKY to have found this used (phone to us & not used hard at '
 'all), phone on line from someone who upgraded and sold this one. My Son '
 'liked his old one that finally fell apart after 2.5+ years and didnt want an '
 'upgrade!! Thank you Seller, we really appreciate it & your honesty re: said '
 'used phone.I recommend this seller very highly & would but from them again!!']


In [8]:
key_words=['screen','size','touch','battery','charge','charger','hours','power','life','camera','light','pictures','photos','front','rear']

In [9]:
for ele in data:
    if len(ele.split())<20:
        data.remove(ele)
    

In [83]:
for sentence in data:
    flag=0
    for word in sentence:
             if word in key_words:
                    flag=1
    if flag==0:
        data.remove(sentence)

In [84]:
print(len(data))

168163


In [85]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [86]:

data_words = list(sent_to_words(data))

print(data_words[:1])

[['nice', 'phone', 'nice', 'up', 'grade', 'from', 'my', 'pantach', 'revue', 'very', 'clean', 'set', 'up', 'and', 'easy', 'set', 'up', 'never', 'had', 'an', 'android', 'phone', 'but', 'they', 'are', 'fantastic', 'to', 'say', 'the', 'least', 'perfect', 'size', 'for', 'surfing', 'and', 'social', 'media', 'great', 'phone', 'samsung']]


In [87]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


In [88]:

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [89]:

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['nice', 'phone', 'nice', 'up', 'grade', 'from', 'my', 'pantach', 'revue', 'very', 'clean', 'set', 'up', 'and', 'easy', 'set', 'up', 'never', 'had', 'an', 'android', 'phone', 'but', 'they', 'are', 'fantastic', 'to', 'say', 'the', 'least', 'perfect', 'size', 'for', 'surfing', 'and', 'social_media', 'great', 'phone', 'samsung']


In [90]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [91]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [92]:

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [93]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ'])

print(data_lemmatized[:1])

[['nice', 'nice', 'grade', 'pantach', 'clean', 'easy', 'set', 'fantastic', 'perfect', 'size', 'great', 'samsung']]


In [94]:
print(len(data_words))

168163


In [95]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [96]:
id2word[0]

'clean'

In [97]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('clean', 1),
  ('easy', 1),
  ('fantastic', 1),
  ('grade', 1),
  ('great', 1),
  ('nice', 2),
  ('pantach', 1),
  ('perfect', 1),
  ('samsung', 1),
  ('set', 1),
  ('size', 1)]]

In [98]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [99]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.081*"love" + 0.059*"bad" + 0.056*"android" + 0.043*"review" + '
  '0.043*"version" + 0.043*"bit" + 0.039*"hand" + 0.038*"much" + 0.026*"sound" '
  '+ 0.025*"awesome"'),
 (1,
  '0.053*"app" + 0.046*"thing" + 0.030*"video" + 0.027*"year" + 0.025*"old" + '
  '0.024*"hot" + 0.024*"compact" + 0.023*"easy" + 0.022*"design" + '
  '0.021*"light"'),
 (2,
  '0.083*"card" + 0.045*"sim" + 0.045*"long" + 0.041*"warranty" + 0.041*"able" '
  '+ 0.038*"money" + 0.034*"return" + 0.032*"network" + 0.024*"store" + '
  '0.020*"part"'),
 (3,
  '0.122*"problem" + 0.090*"nice" + 0.077*"small" + 0.065*"big" + 0.051*"week" '
  '+ 0.046*"many" + 0.033*"side" + 0.033*"right" + 0.029*"memory" + '
  '0.023*"speed"'),
 (4,
  '0.043*"feature" + 0.043*"size" + 0.039*"call" + 0.032*"charge" + '
  '0.032*"case" + 0.027*"charger" + 0.020*"pocket" + 0.018*"full" + '
  '0.017*"text" + 0.016*"touch"'),
 (5,
  '0.159*"good" + 0.126*"great" + 0.062*"phone" + 0.043*"device" + '
  '0.028*"fast" + 0.028*"well" + 0.027

In [101]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.341749759191215

Coherence Score:  0.6097044198029935


In [67]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/nikhithabalagani/ml/env/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.433357 -0.013511       1        1  18.647036
8     -0.000279 -0.198921       2        1  11.755056
0      0.063220 -0.033829       3        1  11.711370
7      0.013998  0.405602       4        1   9.965907
2      0.059242 -0.033796       5        1   9.107787
1      0.063615 -0.026743       6        1   8.900707
3      0.064210 -0.022114       7        1   8.494212
4      0.061897 -0.024261       8        1   8.023774
6      0.055134 -0.028967       9        1   7.468640
5      0.052318 -0.023460      10        1   5.925513, topic_info=     Category         Freq     Term        Total  loglift  logprob
4     Default  3275.000000    great  3275.000000  30.0000  30.0000
23    Default  2475.000000      new  2475.000000  29.0000  29.0000
60    Default  2508.000000     good  2508.000000  28.0000  28.0000
127   Default  1555.000000  product  1555.000000  27.0000  27.0000
54    Default  1531.000000     time  1531.000000  26.0000  26.0000
...       ...          ...      ...          ...      ...      ...
642   Topic10    22.055559  selling    23.250719   2.7731  -5.6115
413   Topic10    21.784960     lock    22.979950   2.7725  -5.6238
2756  Topic10    21.811657   shoddy    23.043869   2.7709  -5.6226
436   Topic10    20.345051     okay    21.540361   2.7688  -5.6922
2855  Topic10    20.572281   unused    21.788019   2.7685  -5.6811

[331 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
68        3  0.996238        able
297       8  0.986906      access
761       9  0.995295   accessory
756       7  0.992401     account
2675      7  0.967378  activation
...     ...       ...         ...
1604      9  0.980463     working
86        4  0.995540       worth
107       5  0.995897       wrong
188       8  0.995975        year
1837      5  0.992183   yesterday

[301 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 1, 8, 3, 2, 4, 5, 7, 6])